In [182]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_transformer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss
from matplotlib import pyplot as plt
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.svm import SVR
from statistics import mean
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
from scipy import stats
import seaborn as sns
import pandas as pd
import numpy as np
import sqlite3
import joblib
import random 
import math

In [183]:
def read_and_encode(file):
    blank = pd.read_csv(file, encoding='ISO-8859-1')
    blank = blank.replace({'Â': '', 'Ã©': 'é', 'Ã': '', '±': 'ñ'}, regex=True)
    return blank

In [184]:
pd.options.mode.chained_assignment = None

players_hof = pd.read_csv('Hall of Fame.csv', encoding='ISO-8859-1')
players_hof = players_hof.replace({'Â': '', 'Ã©': 'é', 'Ã': '', '±': 'ñ'}, regex=True)
retired_players = pd.read_csv('Retired.csv', encoding='ISO-8859-1')
retired_players = retired_players.replace({'Â': '', 'Ã©': 'é', 'Ã': '', '±': 'ñ'}, regex=True)
players_active = pd.read_csv('Active Player Seasons.csv', encoding='ISO-8859-1')
players_active = players_active.replace({'Â': '', 'Ã©': 'é', 'Ã': '', '±': 'ñ'}, regex=True)
all_stars = pd.read_csv('All Stars.csv', encoding='ISO-8859-1')
all_stars = all_stars.replace({'Â': '', 'Ã©': 'é', 'Ã': '', '±': 'ñ'}, regex=True)

as_mvps = pd.read_csv('All Star Game MVP Winners.csv', encoding='utf-8')
al_gg = pd.read_csv('AL Gold Glove Winners.csv', encoding='utf-8')
al_ss = pd.read_csv('AL Silver Slugger Winners.csv', encoding='utf-8')
nl_gg = pd.read_csv('NL Gold Glove Winners.csv', encoding='utf-8')
nl_ss = pd.read_csv('NL Silver Slugger Winners.csv', encoding='utf-8')
bat_titles = pd.read_csv('Batting Title Winners.csv', encoding='utf-8')
roy_winners = pd.read_csv('ROY Winners.csv', encoding='utf-8')
hank_aarons = pd.read_csv('Hank Aaron Award Winners.csv', encoding='utf-8')
mvp_winners = pd.read_csv('MVP Winners.csv', encoding='utf-8')
ps_mvps = pd.read_csv('Post Season MVP Winners.csv', encoding='utf-8')

In [185]:
players_active = players_active.fillna(0)

conn = sqlite3.connect(':memory:')
players_active.to_sql('players_active', conn, index=False, if_exists='replace')

query = """
SELECT *
FROM players_active
WHERE Player IN (
    SELECT Player
    FROM players_active
    GROUP BY Player
    HAVING COUNT(*) >= 4
)
"""

enough_seasons = pd.read_sql_query(query, conn)

conn.close()

enough_seasons

,Rk,Player,WAR,Season,Age,Team,Lg,G,PA,AB,...,WAR.1,WAA,oWAR,dWAR,Rbat,Rdp,Rbaser,Rbaser + Rdp,Rfield,Pos
0,1,Seby Zavala,-0.2,2019,25,CHW,AL,5,12,12,...,-0.2,-0.3,-0.2,0.0,-3,0,0,0,0,/2H
1,2,Seby Zavala,-0.3,2021,27,CHW,AL,37,104,93,...,-0.3,-0.6,0.1,-0.1,-5,0,0,0,-3,2/HD
2,3,Seby Zavala,1.0,2022,28,CHW,AL,61,205,178,...,1.0,0.3,1.1,0.3,2,0,-2,-2,-1,2/H3D
3,4,Seby Zavala,-0.3,2023,29,"ARI,CHW","AL,NL",73,193,175,...,-0.3,-0.9,-0.4,0.5,-13,-1,0,-1,1,2/H1
4,5,Seby Zavala,-0.3,2024,30,SEA,AL,18,43,39,...,-0.3,-0.4,-0.1,-0.1,-3,0,0,0,-2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2987,3906,José Abreu,3.0,2020,33,CHW,AL,60,262,240,...,3.0,2.2,2.5,0.2,22,-1,0,-1,5,*3/D
2988,3907,José Abreu,2.9,2021,34,CHW,AL,152,659,566,...,2.9,0.8,3.0,-0.9,19,-3,1,-3,-1,*3D/5
2989,3908,José Abreu,4.3,2022,35,CHW,AL,157,679,601,...,4.3,2.1,4.2,-0.9,28,-2,1,-1,1,*3D
2990,3909,José Abreu,0.0,2023,36,HOU,AL,141,594,540,...,0.0,-1.9,0.1,-0.8,-8,-1,-2,-4,-1,*3/D


In [186]:
def fix_age(age):
    age_str = str(age)
    if '-' in age:
        return int(age.split('-')[1])
    else:
        return int(age)

players_hof = players_hof.fillna(0)
players_hof['Age'] = players_hof['Age'].apply(fix_age)
hof_age_mean = players_hof['Age'].mean() - 2
hof_age_mean = math.ceil(hof_age_mean)
hof_age_mean

37

In [187]:
count = enough_seasons['Player'].unique()

enough_seasons = enough_seasons.drop(['WAR', 'Lg', 'Pos'], axis=1)
enough_seasons = enough_seasons.rename(columns={'WAR.1': 'WAR'})
new_cols = ['Player', 'Team', 'Age', 'G', 'PA', 'AB',
       'R', 'H', '1B', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'BA',
       'OBP', 'SLG', 'OPS', 'OPS+', 'TB', 'GIDP', 'HBP', 'SH', 'SF', 'IBB',
       'WAR', 'WAA', 'oWAR', 'dWAR', 'Rbat', 'Rdp', 'Rbaser', 'Rbaser + Rdp',
       'Rfield']
enough_seasons = enough_seasons[new_cols]
enough_seasons

,Player,Team,Age,G,PA,AB,R,H,1B,2B,...,IBB,WAR,WAA,oWAR,dWAR,Rbat,Rdp,Rbaser,Rbaser + Rdp,Rfield
0,Seby Zavala,CHW,25,5,12,12,1,1,1,0,...,0,-0.2,-0.3,-0.2,0.0,-3,0,0,0,0
1,Seby Zavala,CHW,27,37,104,93,15,17,9,3,...,0,-0.3,-0.6,0.1,-0.1,-5,0,0,0,-3
2,Seby Zavala,CHW,28,61,205,178,22,48,32,14,...,0,1.0,0.3,1.1,0.3,2,0,-2,-2,-1
3,Seby Zavala,"ARI,CHW",29,73,193,175,17,30,19,4,...,0,-0.3,-0.9,-0.4,0.5,-13,-1,0,-1,1
4,Seby Zavala,SEA,30,18,43,39,4,6,3,2,...,0,-0.3,-0.4,-0.1,-0.1,-3,0,0,0,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2987,José Abreu,CHW,33,60,262,240,43,76,42,15,...,1,3.0,2.2,2.5,0.2,22,-1,0,-1,5
2988,José Abreu,CHW,34,152,659,566,86,148,86,30,...,3,2.9,0.8,3.0,-0.9,19,-3,1,-3,-1
2989,José Abreu,CHW,35,157,679,601,85,183,128,40,...,2,4.3,2.1,4.2,-0.9,28,-2,1,-1,1
2990,José Abreu,HOU,36,141,594,540,62,128,86,23,...,1,0.0,-1.9,0.1,-0.8,-8,-1,-2,-4,-1


In [188]:
def calc_bat_avg(hits, at_bats):
    if at_bats == 0:
        return 0.0
    return round((hits/at_bats), 3)

def calc_obp(hits, walks, hbp, at_bats, sf):
    first = hits + walks + hbp
    second = at_bats + hbp + sf
    if second == 0:
        return 0.0
    return round((first/second), 3)

def calc_slug(singles, doubles, triples, hrs, at_bats):
    if at_bats == 0:
        return 0.0
    doubles = doubles * 2
    triples = triples * 3
    home_bs = hrs * 4
    return round(((singles+doubles+triples+home_bs)/at_bats), 3)

def calc_ops(obp, slug):
    return round((obp+slug), 3)

In [189]:
proj_data = []

early_multiplier = 1.2
prime_multiplier = 1.0
late_multiplier = 0.8

for player in count:
    player_pred = []
    player_pred.append(player)
    player_data = enough_seasons[enough_seasons['Player'] == player]
    sorted_df = player_data.sort_values(by='Age', ascending=True)
    sorted_df = sorted_df.drop(['BA', 'OBP', 'SLG', 'OPS', 'OPS+'], axis=1)
    num_seasons = len(player_data)

            
    if num_seasons < 5:
        multiplier = early_multiplier
    elif num_seasons < 8:
        multiplier = prime_multiplier
    else:
        multiplier = late_multiplier
            
    for j in sorted_df.columns[3:]:
        curr_total = sorted_df[j].sum()
        avg = sorted_df[j].rolling(window=4).mean().iloc[-1]
        std_dev = sorted_df[j].rolling(window=4).std().iloc[-1]
        remaining = hof_age_mean - (sorted_df['Age'].iloc[-1])
        adjustment_factor = 0.5
        proj_num = (avg - (adjustment_factor * std_dev)) * remaining * multiplier
        proj_sum = curr_total + proj_num
        player_pred.append(round(proj_sum))
    proj_data.append(player_pred)

print(proj_data[200:205])

[['Brandon Lowe', 1334, 5354, 4724, 711, 1126, 622, 221, 19, 253, 741, 59, 5, 526, 1412, 2162, 46, 70, 0, 27, 10, 30, 13, 33, -3, 111, 9, 4, 19, -62], ['Michael Lorenzen', 530, 222, 199, 38, 47, 28, 5, 1, 9, 36, 5, 2, 12, 69, 87, 7, 2, 5, 0, 0, 2, 2, 2, 0, -10, -5, -1, -6, 1], ['Nicky Lopez', 1584, 5339, 4749, 582, 1154, 928, 167, 44, 12, 327, 108, 34, 418, 765, 1451, 62, 53, 88, 10, 0, 10, -9, 11, 4, -177, 20, 13, 33, -35], ['Tim Locastro', 554, 1063, 918, 194, 210, 141, 41, 3, 23, 68, 79, 8, 61, 219, 329, 3, 66, 6, 2, 0, -1, -6, -1, -2, -49, 2, 4, 6, -13], ['Francisco Lindor', 2165, 9525, 8432, 1389, 2240, 1364, 434, 37, 386, 1219, 277, 67, 845, 1627, 3926, 141, 109, 31, 78, 39, 75, 45, 68, 19, 200, 13, 27, 42, 54]]


In [190]:
final_cols = ['Player', 'G', 'PA', 'AB',
       'R', 'H', '1B', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'TB', 'GIDP', 'HBP', 'SH', 'SF', 'IBB',
       'WAR', 'WAA', 'oWAR', 'dWAR', 'Rbat', 'Rdp', 'Rbaser', 'Rbaser + Rdp',
       'Rfield']

projection_df = pd.DataFrame(proj_data, columns=final_cols)
war = projection_df.pop('WAR')
projection_df.insert(1, 'WAR', war)
projection_df

,Player,WAR,G,PA,AB,R,H,1B,2B,3B,...,SF,IBB,WAA,oWAR,dWAR,Rbat,Rdp,Rbaser,Rbaser + Rdp,Rfield
0,Seby Zavala,-2,439,1242,1109,134,216,130,44,0,...,7,0,-6,-1,1,-77,-4,-9,-12,-20
1,Christian Yelich,50,1882,8151,7045,1207,2005,1331,377,42,...,31,71,23,52,-10,264,18,43,62,-26
2,Mike Yastrzemski,23,1190,4369,3846,607,902,462,219,30,...,20,15,8,17,2,40,7,0,7,49
3,Austin Wynns,0,356,1016,937,98,214,153,42,0,...,3,0,-4,-1,2,-57,-9,-1,-9,-1
4,Kolten Wong,25,1508,5421,4802,648,1209,840,225,36,...,43,32,7,20,9,-51,16,15,34,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,Jo Adell,-2,809,2856,2646,287,583,345,110,25,...,8,1,-15,-3,-3,-98,-11,-18,-30,14
401,Riley Adams,4,649,1982,1735,152,374,220,89,10,...,6,1,-3,5,2,-33,-11,-19,-19,-14
402,Willy Adames,51,2159,8983,8020,1161,1946,1097,446,7,...,39,25,21,49,9,62,-4,-15,-7,-44
403,Ronald Acuña Jr.,47,1586,7122,6127,1230,1715,1049,320,19,...,34,32,25,49,-10,284,-11,41,37,-34


In [191]:
projection_df['BA'] = projection_df.apply(lambda row: calc_bat_avg(row['H'], row['AB']), axis=1)
projection_df['OBP'] = projection_df.apply(lambda row: calc_obp(row['H'], row['BB'], row['HBP'], row['AB'], row['SF']), axis=1)
projection_df['SLG'] = projection_df.apply(lambda row: calc_slug(row['1B'], row['2B'], row['3B'], row['HR'], row['AB']), axis=1)
projection_df['OPS'] = projection_df.apply(lambda row: calc_ops(row['OBP'], row['SLG']), axis=1)
projection_df

,Player,WAR,G,PA,AB,R,H,1B,2B,3B,...,dWAR,Rbat,Rdp,Rbaser,Rbaser + Rdp,Rfield,BA,OBP,SLG,OPS
0,Seby Zavala,-2,439,1242,1109,134,216,130,44,0,...,1,-77,-4,-9,-12,-20,0.195,0.277,0.312,0.589
1,Christian Yelich,50,1882,8151,7045,1207,2005,1331,377,42,...,-10,264,18,43,62,-26,0.285,0.430,0.455,0.885
2,Mike Yastrzemski,23,1190,4369,3846,607,902,462,219,30,...,2,40,7,0,7,49,0.235,0.353,0.436,0.789
3,Austin Wynns,0,356,1016,937,98,214,153,42,0,...,2,-57,-9,-1,-9,-1,0.228,0.291,0.326,0.617
4,Kolten Wong,25,1508,5421,4802,648,1209,840,225,36,...,9,-51,16,15,34,49,0.252,0.353,0.379,0.732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,Jo Adell,-2,809,2856,2646,287,583,345,110,25,...,-3,-98,-11,-18,-30,14,0.220,0.286,0.387,0.673
401,Riley Adams,4,649,1982,1735,152,374,220,89,10,...,2,-33,-11,-19,-19,-14,0.216,0.321,0.339,0.660
402,Willy Adames,51,2159,8983,8020,1161,1946,1097,446,7,...,9,62,-4,-15,-7,-44,0.243,0.350,0.442,0.792
403,Ronald Acuña Jr.,47,1586,7122,6127,1230,1715,1049,320,19,...,-10,284,-11,41,37,-34,0.280,0.423,0.487,0.910


In [192]:
order_hof_cols = ['Player', 'WAR', 'G', 'PA', 'AB', 'R', 'H', '1B', '2B', '3B', 'HR',
       'RBI', 'SB', 'CS', 'BB', 'SO', 'TB', 'GIDP', 'HBP', 'SH', 'SF', 'IBB',
       'WAA', 'oWAR', 'dWAR', 'Rbat', 'Rdp', 'Rbaser', 'Rbaser + Rdp',
       'Rfield', 'BA', 'OBP', 'SLG', 'OPS']
players_hof = players_hof[order_hof_cols]
players_hof

,Player,WAR,G,PA,AB,R,H,1B,2B,3B,...,dWAR,Rbat,Rdp,Rbaser,Rbaser + Rdp,Rfield,BA,OBP,SLG,OPS
0,Babe Ruth,162.2,2503,10628,8399,2174,2873,1517,506,136,...,-2.3,1327,0,-9,-9,79,0.342,0.474,0.690,1.164
1,Willie Mays,156.2,3005,12545,10924,2068,3293,1967,525,141,...,18.2,804,-8,79,70,185,0.301,0.384,0.557,0.940
2,Ty Cobb,151.4,3034,13103,11440,2245,4189,3053,724,295,...,-10.8,995,0,55,55,0,0.366,0.433,0.512,0.944
3,Henry Aaron,143.1,3298,13941,12364,2174,3771,2294,624,98,...,-4.6,878,-11,44,33,98,0.305,0.374,0.555,0.928
4,Tris Speaker,135.0,2789,12020,10195,1882,3514,2383,792,222,...,2.5,823,0,4,4,92,0.345,0.428,0.500,0.928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,Jesse Haines,-3.0,560,1215,1124,76,209,181,23,2,...,-0.1,-150,0,2,2,0,0.186,0.208,0.218,0.426
252,Red Faber,-3.6,670,1559,1269,98,170,144,21,2,...,0.0,-180,0,3,3,0,0.134,0.240,0.161,0.401
253,Vic Willis,-3.9,531,1632,1493,107,248,224,19,4,...,-0.2,-150,0,3,3,-1,0.166,0.213,0.186,0.399
254,Lefty Grove,-6.5,619,1580,1369,119,202,155,27,5,...,0.1,-232,2,-3,-1,0,0.148,0.209,0.207,0.417


In [193]:
first_column_1 = players_hof.iloc[:, 0]
hof_2 = players_hof.drop(players_hof.columns[0], axis=1)
int_cols_hof = hof_2.columns[:-4]
hof_2[int_cols_hof] = hof_2[int_cols_hof].astype(int)
float_cols_hof = hof_2.columns[-4:]
hof_2[float_cols_hof] = hof_2[float_cols_hof].astype(float)
hof_2.insert(0, 'Player', first_column_1)

first_column_2 = projection_df.iloc[:, 0]
temp_df = projection_df.drop(projection_df.columns[0], axis=1)
int_cols_temp = temp_df.columns[:-4]
temp_df[int_cols_temp] = temp_df[int_cols_temp].astype(int)
float_cols_temp = temp_df.columns[-4:]
temp_df[float_cols_temp] = temp_df[float_cols_temp].astype(float)
temp_df.insert(0, 'Player', first_column_2)

temp_data = pd.merge(hof_2, projection_df, how='outer')
temp_data

,Player,WAR,G,PA,AB,R,H,1B,2B,3B,...,dWAR,Rbat,Rdp,Rbaser,Rbaser + Rdp,Rfield,BA,OBP,SLG,OPS
0,Babe Ruth,162,2503,10628,8399,2174,2873,1517,506,136,...,-2,1327,0,-9,-9,79,0.342,0.474,0.690,1.164
1,Willie Mays,156,3005,12545,10924,2068,3293,1967,525,141,...,18,804,-8,79,70,185,0.301,0.384,0.557,0.940
2,Ty Cobb,151,3034,13103,11440,2245,4189,3053,724,295,...,-10,995,0,55,55,0,0.366,0.433,0.512,0.944
3,Henry Aaron,143,3298,13941,12364,2174,3771,2294,624,98,...,-4,878,-11,44,33,98,0.305,0.374,0.555,0.928
4,Tris Speaker,135,2789,12020,10195,1882,3514,2383,792,222,...,2,823,0,4,4,92,0.345,0.428,0.500,0.928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,Jo Adell,-2,809,2856,2646,287,583,345,110,25,...,-3,-98,-11,-18,-30,14,0.220,0.286,0.387,0.673
657,Riley Adams,4,649,1982,1735,152,374,220,89,10,...,2,-33,-11,-19,-19,-14,0.216,0.321,0.339,0.660
658,Willy Adames,51,2159,8983,8020,1161,1946,1097,446,7,...,9,62,-4,-15,-7,-44,0.243,0.350,0.442,0.792
659,Ronald Acuña Jr.,47,1586,7122,6127,1230,1715,1049,320,19,...,-10,284,-11,41,37,-34,0.280,0.423,0.487,0.910


In [194]:
order_ret_cols = ['Player', 'WAR', 'G', 'PA', 'AB', 'R', 'H', '1B', '2B', '3B', 'HR',
       'RBI', 'SB', 'CS', 'BB', 'SO', 'TB', 'GIDP', 'HBP', 'SH', 'SF', 'IBB',
       'WAA', 'oWAR', 'dWAR', 'Rbat', 'Rdp', 'Rbaser', 'Rbaser + Rdp',
       'Rfield', 'BA', 'OBP', 'SLG', 'OPS']
retired_players = retired_players[order_ret_cols]

first_column_3 = retired_players.iloc[:, 0]
ret_2 = retired_players.drop(retired_players.columns[0], axis=1)
ret_2 = ret_2.fillna(0)
int_cols_ret = ret_2.columns[:-4]
ret_2[int_cols_ret] = ret_2[int_cols_ret].astype(int)
float_cols_ret = ret_2.columns[-4:]
ret_2[float_cols_ret] = ret_2[float_cols_ret].astype(float)
ret_2.insert(0, 'Player', first_column_3)
ret_2 = ret_2[ret_2['G'] >= 1000]
ret_2 = ret_2[ret_2['WAR'] >= 35]
merged_data = pd.merge(temp_data, ret_2, how='outer')
merged_data

,Player,WAR,G,PA,AB,R,H,1B,2B,3B,...,dWAR,Rbat,Rdp,Rbaser,Rbaser + Rdp,Rfield,BA,OBP,SLG,OPS
0,Babe Ruth,162,2503,10628,8399,2174,2873,1517,506,136,...,-2,1327,0,-9,-9,79,0.342,0.474,0.690,1.164
1,Willie Mays,156,3005,12545,10924,2068,3293,1967,525,141,...,18,804,-8,79,70,185,0.301,0.384,0.557,0.940
2,Ty Cobb,151,3034,13103,11440,2245,4189,3053,724,295,...,-10,995,0,55,55,0,0.366,0.433,0.512,0.944
3,Henry Aaron,143,3298,13941,12364,2174,3771,2294,624,98,...,-4,878,-11,44,33,98,0.305,0.374,0.555,0.928
4,Tris Speaker,135,2789,12020,10195,1882,3514,2383,792,222,...,2,823,0,4,4,92,0.345,0.428,0.500,0.928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,Mickey Vernon,35,2409,9839,8731,1196,2495,1713,490,120,...,-13,176,9,-5,4,-21,0.286,0.359,0.428,0.787
853,Edwin Encarnación,35,1960,8126,7040,1099,1832,1028,370,10,...,-15,243,-14,-7,-20,-69,0.260,0.350,0.496,0.846
854,Eddie Yost,35,2109,9177,7346,1215,1863,1331,337,56,...,-10,190,-5,8,4,-113,0.254,0.394,0.371,0.765
855,Brady Anderson,35,1834,7737,6499,1062,1661,1046,338,67,...,-3,97,15,27,42,-28,0.256,0.362,0.425,0.787


In [195]:
merged_data["status"] = np.nan 

hof_check = []
for i in range(len(players_hof)):
    star = players_hof.at[i,'Player']
    hof_check.append(star)

retired_check = []
for i in range(len(retired_players)):
    star = retired_players.at[i,'Player']
    retired_check.append(star)

for i in range(len(merged_data)):
    player = merged_data.at[i,'Player']
    if player in hof_check:
        merged_data.at[i,'status'] = 'hof'
    elif player in retired_check:
        merged_data.at[i,'status'] = 'retired'
    else:
        merged_data.at[i,'status'] = 'active'
        
merged_data

,Player,WAR,G,PA,AB,R,H,1B,2B,3B,...,Rbat,Rdp,Rbaser,Rbaser + Rdp,Rfield,BA,OBP,SLG,OPS,status
0,Babe Ruth,162,2503,10628,8399,2174,2873,1517,506,136,...,1327,0,-9,-9,79,0.342,0.474,0.690,1.164,hof
1,Willie Mays,156,3005,12545,10924,2068,3293,1967,525,141,...,804,-8,79,70,185,0.301,0.384,0.557,0.940,hof
2,Ty Cobb,151,3034,13103,11440,2245,4189,3053,724,295,...,995,0,55,55,0,0.366,0.433,0.512,0.944,hof
3,Henry Aaron,143,3298,13941,12364,2174,3771,2294,624,98,...,878,-11,44,33,98,0.305,0.374,0.555,0.928,hof
4,Tris Speaker,135,2789,12020,10195,1882,3514,2383,792,222,...,823,0,4,4,92,0.345,0.428,0.500,0.928,hof
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852,Mickey Vernon,35,2409,9839,8731,1196,2495,1713,490,120,...,176,9,-5,4,-21,0.286,0.359,0.428,0.787,retired
853,Edwin Encarnación,35,1960,8126,7040,1099,1832,1028,370,10,...,243,-14,-7,-20,-69,0.260,0.350,0.496,0.846,retired
854,Eddie Yost,35,2109,9177,7346,1215,1863,1331,337,56,...,190,-5,8,4,-113,0.254,0.394,0.371,0.765,retired
855,Brady Anderson,35,1834,7737,6499,1062,1661,1046,338,67,...,97,15,27,42,-28,0.256,0.362,0.425,0.787,retired


In [196]:
hof_temp = pd.read_csv('Hall of Fame.csv', encoding='ISO-8859-1')
hof_temp = hof_temp.replace({'Â': '', 'Ã©': 'é', 'Ã': '', '±': 'ñ'}, regex=True)
hof_temp.set_index(hof_temp.columns[0], inplace=True)

ret_temp = pd.read_csv('Retired.csv', encoding='ISO-8859-1')
ret_temp = ret_temp.replace({'Â': '', 'Ã©': 'é', 'Ã': '', '±': 'ñ'}, regex=True)
ret_temp.set_index(ret_temp.columns[0], inplace=True)

act_temp = pd.read_csv('Active Player Seasons.csv', encoding='ISO-8859-1')
act_temp = act_temp.replace({'Â': '', 'Ã©': 'é', 'Ã': '', '±': 'ñ'}, regex=True)
act_temp.set_index(act_temp.columns[0], inplace=True)

In [197]:
def check_active(act_temp, year, glover):
    for i in act_temp.index:
        player = act_temp.loc[i]
        name = player['Player'].split()
        if name[1] == glover[0] and int(player['Season']) == int(year):
            if player['Team'] == glover[1]:
                return player['Player']
    return None

def check_hof(hof_temp, year, glover):
    for i in hof_temp.index:
        player = hof_temp.loc[i]
        name = player['Player'].split()
        if name[1] == glover[0] and glover[1] in player['Team'].split(','):
            if int(player['From']) <= int(year) <= int(player['To']):
                return player['Player']
    return None

def check_retired(ret_temp, year, glover):
    for i in ret_temp.index:
        player = ret_temp.loc[i]
        name = player['Player'].split()
        if name[1] == glover[0] and glover[1] in player['Team'].split(','):
            if int(player['From']) <= int(year) <= int(player['To']):
                return player['Player']
    return None

def setup_temps(df):
    df = df.copy()
    df.set_index(df.columns[0], inplace=True)
    team = 'Team'
    if team in df.columns:
        df.drop('Team', axis=1, inplace=True)
    return df

def gg_ss_checks(league, whole, col):
    for i in league.index:
        for j in league.columns:
            if pd.isna(league.loc[i, j]):
                continue
            glover = league.loc[i,j].split(',')
            glover = [item.strip('\xa0') for item in glover]
            year = i.split()
            check_1 = check_active(year[0], glover)
            check_2 = check_hof(year[0], glover)
            check_3 = check_retired(year[0], glover)
            if check_1:
                whole.loc[whole['Player'] == check_1, col] += 1
            if check_2:
                whole.loc[whole['Player'] == check_2, col] += 1
            if check_3:
                whole.loc[whole['Player'] == check_3, col] += 1

def simple_awards(df, award_df, award, count=False):
    df[award] = 0
    if count == False:
        df[award] = False
    award_df_2 = award_df.copy()

    for i in award_df_2.index:
        player = award_df_2.iloc[i, 2]
        for j in df.index:
            temp = df.loc[j, 'Player']
            if temp == player:
                if count == False:
                    df.loc[j, award] = True
                else:
                    df.loc[j, award] += 1

In [198]:
merged_data['GGs'] = 0
al_gg_temp = setup_temps(al_gg)
nl_gg_temp = setup_temps(nl_gg)

gg_ss_checks(al_gg_temp, merged_data, 'GGs')
gg_ss_checks(nl_gg_temp, merged_data, 'GGs')

In [199]:
merged_data['SSs'] = 0
al_ss_temp = setup_temps(al_ss)
nl_ss_temp = setup_temps(nl_ss)

gg_ss_checks(al_ss_temp, merged_data, 'SSs')
gg_ss_checks(nl_ss_temp, merged_data, 'SSs')

In [200]:
simple_awards(merged_data, roy_winners, 'ROY', count=False)
simple_awards(merged_data, hank_aarons, 'Hank_Aaron', count=True)
simple_awards(merged_data, mvp_winners, 'MVPs', count=True)

In [201]:
merged_data['PS_MVPs'] = 0
ps_2 = ps_mvps.copy()
ps_2.set_index(ps_2.columns[0], inplace=True)

for i in ps_2.index:
    for j in ps_2.columns:
        if pd.isna(ps_2.loc[i, j]):
            continue
        player = ps_2.loc[i, j]
        if ',' in player:
            blank = player.split()
            player_1 = blank[0]
            player_2 = blank[1]
            blank = [player_1, player_2]
            for count in blank:
                for x in merged_data.index:
                    temp = merged_data.loc[x, 'Player']
                    if temp == count:
                        merged_data.loc[x, 'PS_MVPs'] += 1  
        else:
            for y in merged_data.index:
                    temp = merged_data.loc[y, 'Player']
                    if temp == player:
                        merged_data.loc[y, 'PS_MVPs'] += 1 

In [202]:
merged_data['Bat_Titles'] = 0
bt_2 = bat_titles.copy()
bt_2['Batting Champ'] = bt_2['Batting Champ'].str.replace('\xa0', ' ')

simple_awards(merged_data, bt_2, 'Bat_Titles', count=True)

In [203]:
as_2 = all_stars.copy()
as_2.set_index(as_2.columns[0], inplace=True)
as_2 = as_2.reset_index(drop=True)

simple_awards(merged_data, as_2, 'All_Stars', count=True)

In [204]:
col_1 = merged_data.pop('ROY')
merged_data.insert(35, 'ROY', col_1)
col_2 = merged_data.pop('MVPs')
merged_data.insert(36, 'MVPs', col_2)
col_3 = merged_data.pop('PS_MVPs')
merged_data.insert(37, 'PS_MVPs', col_3)

In [205]:
award_age = hof_age_mean - 1

early_multiplier = 1.2
prime_multiplier = 1.1
late_multiplier = 0.8

award_multipliers = {
    'GGs': 0.8,
    'SSs': 1.0,
    'Hank_Aaron': 0.8,
    'Bat_Titles': 0.6,
    'All_Stars': 1.1
}

for i in merged_data.index:
    if merged_data.loc[i, 'status'] == 'active':
        player = merged_data.loc[i, 'Player']
        seasons = act_temp[act_temp['Player'] == player]
        num_seasons = len(seasons)
        multiplier = 0
        
        if num_seasons < 5:
            multiplier = early_multiplier
        elif num_seasons < 8:
            multiplier = prime_multiplier
        else:
            multiplier = late_multiplier
        
        for j in merged_data.columns[38:]:
            col_multiplier = award_multipliers.get(j, 1.0)
            curr_total = merged_data.loc[i, j]
            avg = int(curr_total) / num_seasons
            std_dev = curr_total / (num_seasons * (12**0.5))
            remaining = hof_age_mean - (seasons['Age'].max())
            adjustment_factor = 0.25
            proj_num = (avg - (adjustment_factor * std_dev)) * remaining * multiplier * col_multiplier
            proj_num = curr_total + proj_num
            merged_data.loc[i, j] = round(proj_num)

In [206]:
def log_transform(X):
    X = X.clip(min=1e-8)
    return np.log(X)

cat_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"), OneHotEncoder(handle_unknown="ignore"))

log_pipeline = make_pipeline(SimpleImputer(strategy="median"), FunctionTransformer(log_transform, feature_names_out="one-to-one"), StandardScaler())

default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

preprocessing = ColumnTransformer([
    ("cat", cat_pipeline, make_column_selector(dtype_include=object)),
    ("log", log_pipeline, make_column_selector(dtype_include=np.number)),
    
],
remainder= default_num_pipeline)

models = {
    "Linear Regression": make_pipeline(preprocessing, LinearRegression()),
    "Ridge Regression": make_pipeline(preprocessing, Ridge()),
    "Lasso Regression": make_pipeline(preprocessing, Lasso()),
    "Random Forest Regressor": make_pipeline(preprocessing, RandomForestRegressor(random_state=42)),
    "Random Forest Classifier": make_pipeline(preprocessing, RandomForestClassifier(random_state=42)),
    "Gradient Boosting Regressor": make_pipeline(preprocessing, GradientBoostingRegressor(random_state=42)),
    "Support Vector Regressor": make_pipeline(preprocessing, SVR()),
    "K-Nearest Neighbors": make_pipeline(preprocessing, KNeighborsRegressor()),
}

In [207]:
x_cols = ['WAR', 'G', 'PA', 'AB', 'R', 'H', '1B', '2B', '3B', 'HR',
       'RBI', 'SB', 'CS', 'BB', 'SO', 'TB', 'GIDP', 'HBP', 'SH', 'SF', 'IBB',
       'WAA', 'oWAR', 'dWAR', 'Rbat', 'Rdp', 'Rbaser', 'Rbaser + Rdp',
       'Rfield', 'BA', 'OBP', 'SLG', 'OPS', 'status', 'ROY', 'MVPs', 'PS_MVPs',
       'GGs', 'SSs', 'Hank_Aaron', 'Bat_Titles', 'All_Stars']

y_col = 'status'

In [208]:
x_data = merged_data[x_cols].copy()
x_data = x_data.fillna(0)

y_data = merged_data[y_col].copy()
label_encoder = LabelEncoder()
y_data = label_encoder.fit_transform(y_data)

results = {}

for model_name, model in models.items():
    
    model_rmses = -cross_val_score(model, x_data, y_data, scoring="neg_root_mean_squared_error", cv=10)
    
    results[model_name] = {
        "RMSE": np.mean(model_rmses)
    }
    
results_df = pd.DataFrame(results)
results_df.head()

,Linear Regression,Ridge Regression,Lasso Regression,Random Forest Regressor,Random Forest Classifier,Gradient Boosting Regressor,Support Vector Regressor,K-Nearest Neighbors
RMSE,3.041712e-15,0.009961,0.788891,0.0,0.0,0.000021,0.215603,0.560976


In [209]:
curr_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("scaler", StandardScaler()),
    ("random_forest", RandomForestClassifier(random_state=42))
])

random_search = RandomizedSearchCV(
    curr_pipeline, 
    param_distributions={
        "random_forest__n_estimators": randint(100, 300),
        "random_forest__max_depth": [None, 10, 20, 30],
        "random_forest__min_samples_split": [2, 5, 10],
        "random_forest__min_samples_leaf": [1, 2, 4],
        "random_forest__max_features": ["sqrt", "log2", None]
    },
    n_iter=50,
    cv=10,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)

random_search.fit(x_data, y_data)

RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('preprocessing',
                                              ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                           SimpleImputer(strategy='median')),
                                                                                          ('standardscaler',
                                                                                           StandardScaler())]),
                                                                transformers=[('cat',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer(strategy='most_frequent')),
                                                                                               ('onehotencoder',
                                                                                                OneHotEncoder(handle_unknown='ignore'...
                   param_distributions={'random_forest__max_depth': [None, 10,
                                                                     20, 30],
                                        'random_forest__max_features': ['sqrt',
                                                                        'log2',
                                                                        None],
                                        'random_forest__min_samples_leaf': [1,
                                                                            2,
                                                                            4],
                                        'random_forest__min_samples_split': [2,
                                                                             5,
                                                                             10],
                                        'random_forest__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000255BFCDD910>},
                   scoring='neg_root_mean_squared_error')

In [210]:
best_params = random_search.best_params_

best_rf = RandomForestClassifier(
    max_depth=best_params['random_forest__max_depth'],
    max_features=best_params['random_forest__max_features'],
    min_samples_leaf=best_params['random_forest__min_samples_leaf'],
    min_samples_split=best_params['random_forest__min_samples_split'],
    n_estimators=best_params['random_forest__n_estimators'],
    class_weight='balanced',
    random_state=42  
)

final_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("scaler", StandardScaler()),
    ("random_forest", best_rf)
])

rf_data = merged_data.copy()
rf_data['status_encoded'] = label_encoder.fit_transform(rf_data['status'])
new_y_col = 'status_encoded'

train = rf_data[rf_data['status'] != 'active']
test = rf_data[rf_data['status'] == 'active']
final_pipeline.fit(train[x_cols], train[new_y_col])
predictions = final_pipeline.predict(test[x_cols])
predictions = predictions.round().astype(int)
predictions_df = pd.Series(predictions, index=test.index, name='predictions')
rf_data['predictions'] = predictions_df

In [211]:
hof_predictions = rf_data[(rf_data['predictions'] == 1) & (test['WAR'] >= 56)]
hof_predictions

,Player,WAR,G,PA,AB,R,H,1B,2B,3B,...,ROY,MVPs,PS_MVPs,GGs,SSs,Hank_Aaron,Bat_Titles,All_Stars,status_encoded,predictions
293,Kyle Tucker,74,1765,7305,6318,1045,1762,975,366,27,...,False,0,0,2,2,0,0,8,0,1.0
295,Mike Trout,94,1700,7422,6164,1239,1826,999,339,59,...,True,3,0,0,11,2,0,14,0,1.0
312,Fernando Tatis Jr.,62,1593,6900,6142,1140,1698,924,336,17,...,False,0,0,0,0,0,0,7,0,1.0
324,Juan Soto,107,2776,12125,9578,1847,2650,1529,469,34,...,False,0,0,0,11,0,2,12,0,1.0
340,Marcus Semien,63,2015,8849,7984,1240,2024,1230,407,43,...,False,0,0,1,2,0,0,4,0,1.0
342,Corey Seager,60,1648,7149,6347,1014,1851,1106,376,15,...,True,0,3,0,5,0,0,8,0,1.0
365,Austin Riley,66,2066,8826,7912,1218,2162,1243,446,26,...,False,0,0,0,5,0,0,6,0,1.0
373,José Ramírez,80,2192,9258,8165,1347,2263,1245,533,57,...,False,0,0,0,7,0,0,8,0,1.0
398,Matt Olson,56,1951,8304,7154,1102,1822,936,390,7,...,False,0,0,3,2,0,0,3,0,1.0
400,Shohei Ohtani,66,2028,8650,7390,1345,2069,1017,384,90,...,True,3,0,0,6,4,0,9,0,1.0
